In [1]:
import tensorflow as tf
import os
import sys
sys.path.append('./bert')
import numpy as np
import transformer
import embedding_layer
import model_utils
from collections import defaultdict
import collections
import summary_model
import lm_model

/Users/higgs/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
max_len = 64
inp = tf.placeholder(tf.int32, shape=[None, max_len], name='inp')
inp_len = tf.placeholder(tf.int32, shape=[None], name='inp_len')
config = summary_model.BASE_PARAMS
config.keys()

['vocab_size',
 'num_hidden_layers',
 'default_batch_size',
 'label_smoothing',
 'default_batch_size_tpu',
 'filter_size',
 'static_batch',
 'learning_rate_decay_rate',
 'alpha',
 'allow_ffn_pad',
 'num_types',
 'use_tpu',
 'attention_dropout',
 'layer_postprocess_dropout',
 'relu_dropout',
 'learning_rate_warmup_steps',
 'optimizer_adam_beta1',
 'optimizer_adam_beta2',
 'num_heads',
 'beam_size',
 'max_length',
 'extra_decode_length',
 'initializer_gain',
 'initializer_range',
 'hidden_size',
 'optimizer_adam_epsilon',
 'learning_rate']

In [3]:
config['batch_size'] = 4

In [4]:
config['vocab_size']

32003

In [5]:
config['max_length']

200

In [6]:
import tokenization

In [7]:
tokenizer = tokenization.FullTokenizer('./model/vocab.txt')

In [8]:
len(tokenizer.vocab)

21128

In [9]:
config['vocab_size'] = len(tokenizer.vocab)
config['max_length'] = 64

In [10]:
transformer_m = transformer.Transformer(params=config, train=True)

In [11]:
attention_bias = model_utils.get_decoder_self_attention_bias(config['max_length'])

In [12]:
encoder_outputs = transformer_m.encode(inp, attention_bias)

In [13]:
logits = transformer_m.embedding_softmax_layer.linear(encoder_outputs)

In [14]:
logits

<tf.Tensor 'presoftmax_linear/Reshape_1:0' shape=(?, ?, 21128) dtype=float32>

In [15]:
weights = tf.sequence_mask(inp_len, config['max_length'], dtype=tf.int32)

In [16]:
weights

<tf.Tensor 'SequenceMask/Cast_1:0' shape=(?, 64) dtype=int32>

In [17]:
loss = model_utils.soft_cross_entropy_loss(logits, inp, config['label_smoothing'], config['vocab_size'])

In [18]:
loss = tf.to_float(weights)

In [19]:
loss = tf.reduce_sum(loss, axis=1)

In [20]:
loss = loss/tf.to_float(inp_len)

In [21]:
mean_loss = tf.reduce_mean(loss)

In [22]:
# class LMModel:
#     def __init__(self, config, max_len):
#         self.config = config
#         self.max_len = max_len
#         self.inp = tf.placeholder(tf.int32, shape=[None, max_len], name='inp')
#         self.inp_len = tf.placeholder(tf.int32, shape=[None], name='inp_len')
    
#     def loss(self, is_training):
#         self.transformer = transformer.Transformer(self.config, is_training)
#         self.attention_bias = model_utils.get_decoder_self_attention_bias(self.max_len)
#         encoder_outputs = self.transformer.encode(self.inp, self.attention_bias)
#         logits = self.transformer.embedding_softmax_layer.linear(encoder_outputs)
#         loss = model_utils.soft_cross_entropy_loss(logits, self.inp, self.config['label_smoothing'], self.config['vocab_size'])
#         weights = tf.sequence_mask(self.inp_len, self.max_len, dtype=tf.int32)
#         loss = loss * tf.to_float(weights)
#         loss = tf.reduce_sum(loss, axis=1)
#         loss = loss/tf.to_float(self.inp_len)
#         loss = tf.reduce_mean(loss)
#         return loss

In [23]:
lm = lm_model.LMModel(config, config['max_length'])

In [24]:
loss = lm.loss(True)